#### **Ingesta del archivo "language.csv"**

In [0]:
dbutils.widgets.text("environment","production")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-16")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit, current_timestamp

##### Esquema

In [0]:
df_language_schema = StructType([
    StructField("languageId",IntegerType(),False),
    StructField("languageCode",StringType(),True),
    StructField("languageName",StringType(),True),
])

##### Leer dataframe

In [0]:
df_language = spark.read.option("header",True).schema(df_language_schema).csv(f"{bronze_folder_path}/{var_file_date }/language.csv")

##### Seleccionar columnas

In [0]:
df_language_select = df_language.select(df_language.languageId,df_language.languageName)

##### Renombrar columnas

In [0]:
df_language_renamed = df_language_select.withColumnsRenamed({"languageId":"language_id","languageName":"language_name"})

##### Agregar columnas

In [0]:
df_final = add_ingestion_date(df_language_renamed).withColumn("environment",lit(var_environment)).withColumn("file_date",lit(var_file_date))

##### Escribir dataframe en Silver

In [0]:
df_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT * from movie_silver.languages